In [143]:
import numpy as np
import glob2 as glob
import pandas as pd
import copy
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import json

In [155]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 10)

    def forward(self, x):
        x = nn.Flatten()(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.softmax(x, dim=1)
        return output

In [156]:
files = glob.glob('1K_model_weights_3_layers/*.npy')

weights_dict = {'bias' : {}, 'weight' : {}}

for file in files:
    layer_name = file.split('/')[-1].split('.')[0]
    if layer_name.split('_')[-1] == 'bias':
        weights_dict['bias'].update({layer_name.split('_')[0]: np.load(file)})
    else:
        weights_dict['weight'].update({layer_name.split('_')[0]: np.load(file)})

order= ['fc1', 'fc2', 'fc3']

transform = transforms.Compose([
    transforms.ToTensor()
])

dataset_test = datasets.MNIST('../data', train=False, transform = transform, download=True)
subset = torch.utils.data.Subset(dataset_test, np.random.choice(len(dataset_test), 5000, replace=False))

In [537]:
def choose_neurons_numeric(chosen_neurons, _weights_dict):
    
    weights_dict_ = copy.deepcopy(_weights_dict)
        
    weights_dict_['weight']['fc1'] = weights_dict_['weight']['fc1'][chosen_neurons[0], :]
    weights_dict_['bias']['fc1'] = weights_dict_['bias']['fc1'][chosen_neurons[0]]

    weights_dict_['weight']['fc2'] = weights_dict_['weight']['fc2'][chosen_neurons[1], :][:, chosen_neurons[0]]
    weights_dict_['bias']['fc2'] = weights_dict_['bias']['fc2'][chosen_neurons[1]]

    weights_dict_['weight']['fc3'] = weights_dict_['weight']['fc3'][: ,chosen_neurons[1]]
    
    return weights_dict_

def load_new_model(weights_dict_):
    model = Net()
    
    model.fc1.weight.data = torch.tensor(weights_dict_['weight']['fc1'])
    model.fc1.bias.data = torch.tensor(weights_dict_['bias']['fc1'])
    model.fc2.weight.data = torch.tensor(weights_dict_['weight']['fc2'])
    model.fc2.bias.data = torch.tensor(weights_dict_['bias']['fc2'])
    model.fc3.weight.data = torch.tensor(weights_dict_['weight']['fc3'])
    model.fc3.bias.data = torch.tensor(weights_dict_['bias']['fc3'])

    return model
    
def print_neurons_shape(weights_dict_):
    print('Layer Number\t|\t Weights Shape\t|\t Bias Shape')
    print('-'*60)
    print('Layer 1  \t|\t', weights_dict_['weight']['fc1'].shape, '\t|\t' , weights_dict_['bias']['fc1'].shape)
    print('Layer 2  \t|\t', weights_dict_['weight']['fc2'].shape, '\t|\t' , weights_dict_['bias']['fc2'].shape)
    print('Layer 3  \t|\t', weights_dict_['weight']['fc3'].shape, '\t|\t' , weights_dict_['bias']['fc3'].shape)


def apply_random_change(arr, X):
    arr_ = copy.deepcopy(arr)

    ones = np.where(arr_ == 1)[0]
    X = int(len(ones) * X)

    to_zeros = np.random.choice(ones, X, replace = False)

    arr_[to_zeros] = 0

    return arr_


def CE_loss(output, target):
    # This takes a batch of 64  , and returns the average loss
    zeros = torch.zeros_like(output)
    zeros[range(output.shape[0]), target] = 1
    # print(output)
    log_probs = torch.log(output + 1e-8)
    return -torch.sum(zeros * log_probs) / output.shape[0]

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = CE_loss(output, target)
            test_loss += loss

    test_loss /= len(test_loader.dataset)

    return test_loss
    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     test_loss, correct, len(test_loader.dataset),
    #     100. * correct / len(test_loader.dataset)))

print_neurons_shape(weights_dict)

Layer Number	|	 Weights Shape	|	 Bias Shape
------------------------------------------------------------
Layer 1  	|	 (1024, 784) 	|	 (1024,)
Layer 2  	|	 (1024, 1024) 	|	 (1024,)
Layer 3  	|	 (10, 1024) 	|	 (10,)


In [446]:
### VALIDATION DRAFT: DO NOT REMOVE AND DO NOT RUN!!! ###

# # chosen_fc1 = np.random.randint(0, 2, 2048)
# # chosen_fc2 = np.random.randint(0, 2, 2048)

# # print(chosen_fc1.sum(), chosen_fc2.sum())

# # expressioned_weights = choose_neurons_expression([chosen_fc1, chosen_fc2], weights_dict)

# # 1012 1032

# print((expressioned_weights['weight']['fc1'].sum(axis = 0) != 0).sum() , (expressioned_weights['weight']['fc1'].sum(axis = 1) != 0).sum())
# print((expressioned_weights['weight']['fc2'].sum(axis = 0) != 0).sum() , (expressioned_weights['weight']['fc2'].sum(axis = 1) != 0).sum())
# print((expressioned_weights['weight']['fc3'].sum(axis = 0) != 0).sum() , (expressioned_weights['weight']['fc3'].sum(axis = 1) != 0).sum())

### DoCplex

In [540]:
from docplex.mp.model import Model

options_per_step = 64 # Lower is faster but less accurate
random_change_perc = 0.01 # Think of it as learning rate.
iterations = 100

test_loader = torch.utils.data.DataLoader(subset, batch_size = 64)

In [541]:
np.random.seed(41)

layer1_chosen_neurons = np.ones(1024)
layer2_chosen_neurons = np.ones(1024)

for iteration in range(iterations):
    m = Model()

    X = m.binary_var_list(options_per_step, name='option')

    layer1_options = [apply_random_change(layer1_chosen_neurons, random_change_perc) for _ in range(options_per_step)]
    layer2_options = [apply_random_change(layer2_chosen_neurons, random_change_perc) for _ in range(options_per_step)]

    loss_per_option = []

    for i in tqdm(range(options_per_step)):
        expressioned_weights = choose_neurons_numeric([layer1_options[i].astype(bool), layer2_options[i].astype(bool)], weights_dict)
        
        neural_model = load_new_model(expressioned_weights)
        loss = test(neural_model, 'cpu', test_loader)
        loss_per_option.append(loss.item())

    m.add_constraint(m.sum(X) == 1)

    m.minimize(m.sum([X[i] * loss_per_option[i] for i in range(options_per_step)]))
    solution = m.solve()
    best_index = int(json.loads(solution.export_as_json_string())['CPLEXSolution']['variables'][0]['index'])

    layer1_chosen_neurons = layer1_options[best_index]
    layer2_chosen_neurons = layer2_options[best_index]

    print(f'Best Objective Value on Iteration {iteration}: {solution.get_objective_value()}')
    print(f'Best Index on Iteration {iteration}: {best_index}')
    print(f'Number of Neurons in layer 1 on Iteration {iteration}: {layer1_chosen_neurons.sum()}')
    print(f'Number of Neurons in layer 2 on Iteration {iteration}: {layer2_chosen_neurons.sum()}')
    print('='*80)

  0%|          | 0/64 [00:00<?, ?it/s]

100%|██████████| 64/64 [01:10<00:00,  1.10s/it]

Best Objective Value on Iteration 0: 0.003250337205827236
Best Index on Iteration 0: 38
Number of Neurons in layer 1 on Iteration 0: 1014.0
Number of Neurons in layer 2 on Iteration 0: 1014.0



100%|██████████| 64/64 [01:03<00:00,  1.01it/s]

Best Objective Value on Iteration 1: 0.0032602252904325724
Best Index on Iteration 1: 56
Number of Neurons in layer 1 on Iteration 1: 1004.0
Number of Neurons in layer 2 on Iteration 1: 1004.0


...

Best Objective Value on Iteration 67: 0.010064566507935524
Best Index on Iteration 67: 21
Number of Neurons in layer 1 on Iteration 67: 543.0
Number of Neurons in layer 2 on Iteration 67: 543.0



100%|██████████| 64/64 [00:58<00:00,  1.09it/s]

Best Objective Value on Iteration 68: 0.010268873535096645
Best Index on Iteration 68: 58
Number of Neurons in layer 1 on Iteration 68: 538.0
Number of Neurons in layer 2 on Iteration 68: 538.0



 88%|████████▊ | 56/64 [00:53<00:07,  1.05it/s]


KeyboardInterrupt: 

 0%|          | 0/64 [00:00<?, ?it/s]
100%|██████████| 64/64 [01:10<00:00,  1.10s/it]
Best Objective Value on Iteration 0: 0.003250337205827236
Best Index on Iteration 0: 38
Number of Neurons in layer 1 on Iteration 0: 1014.0
Number of Neurons in layer 2 on Iteration 0: 1014.0
________________________________

100%|██████████| 64/64 [01:03<00:00,  1.01it/s]
Best Objective Value on Iteration 1: 0.0032602252904325724
Best Index on Iteration 1: 56
Number of Neurons in layer 1 on Iteration 1: 1004.0
Number of Neurons in layer 2 on Iteration 1: 1004.0
________________________________

100%|██████████| 64/64 [01:01<00:00,  1.04it/s]
Best Objective Value on Iteration 2: 0.0032757630106061697
Best Index on Iteration 2: 38
Number of Neurons in layer 1 on Iteration 2: 994.0
Number of Neurons in layer 2 on Iteration 2: 994.0
________________________________

100%|██████████| 64/64 [01:08<00:00,  1.07s/it]
Best Objective Value on Iteration 3: 0.0032864841632544994
Best Index on Iteration 3: 38
Number of Neurons in layer 1 on Iteration 3: 985.0
Number of Neurons in layer 2 on Iteration 3: 985.0
________________________________

100%|██████████| 64/64 [01:01<00:00,  1.04it/s]
Best Objective Value on Iteration 4: 0.0033089725766330957
Best Index on Iteration 4: 7
Number of Neurons in layer 1 on Iteration 4: 976.0
Number of Neurons in layer 2 on Iteration 4: 976.0
________________________________

100%|██████████| 64/64 [01:01<00:00,  1.04it/s]
Best Objective Value on Iteration 5: 0.0033253540750592947
Best Index on Iteration 5: 50
Number of Neurons in layer 1 on Iteration 5: 967.0
Number of Neurons in layer 2 on Iteration 5: 967.0
________________________________

100%|██████████| 64/64 [01:07<00:00,  1.06s/it]
Best Objective Value on Iteration 6: 0.003354686079546809
Best Index on Iteration 6: 15
Number of Neurons in layer 1 on Iteration 6: 958.0
Number of Neurons in layer 2 on Iteration 6: 958.0
________________________________

100%|██████████| 64/64 [01:13<00:00,  1.14s/it]
Best Objective Value on Iteration 7: 0.0033849431201815605
Best Index on Iteration 7: 8
Number of Neurons in layer 1 on Iteration 7: 949.0
Number of Neurons in layer 2 on Iteration 7: 949.0
________________________________

100%|██████████| 64/64 [01:02<00:00,  1.02it/s]
Best Objective Value on Iteration 8: 0.003417019732296467
Best Index on Iteration 8: 59
Number of Neurons in layer 1 on Iteration 8: 940.0
Number of Neurons in layer 2 on Iteration 8: 940.0
________________________________

100%|██████████| 64/64 [01:03<00:00,  1.00it/s]
Best Objective Value on Iteration 9: 0.0034545580856502056
Best Index on Iteration 9: 34
Number of Neurons in layer 1 on Iteration 9: 931.0
Number of Neurons in layer 2 on Iteration 9: 931.0
________________________________

100%|██████████| 64/64 [01:02<00:00,  1.03it/s]
Best Objective Value on Iteration 10: 0.0034880656749010086
Best Index on Iteration 10: 43
Number of Neurons in layer 1 on Iteration 10: 922.0
Number of Neurons in layer 2 on Iteration 10: 922.0
________________________________

100%|██████████| 64/64 [01:02<00:00,  1.02it/s]
Best Objective Value on Iteration 11: 0.0035304538905620575
Best Index on Iteration 11: 26
Number of Neurons in layer 1 on Iteration 11: 913.0
Number of Neurons in layer 2 on Iteration 11: 913.0
________________________________

100%|██████████| 64/64 [01:03<00:00,  1.02it/s]
Best Objective Value on Iteration 12: 0.0035760158207267523
Best Index on Iteration 12: 61
Number of Neurons in layer 1 on Iteration 12: 904.0
Number of Neurons in layer 2 on Iteration 12: 904.0
________________________________

100%|██████████| 64/64 [01:02<00:00,  1.03it/s]
Best Objective Value on Iteration 13: 0.003626028774306178
Best Index on Iteration 13: 40
Number of Neurons in layer 1 on Iteration 13: 895.0
Number of Neurons in layer 2 on Iteration 13: 895.0
________________________________

100%|██████████| 64/64 [01:00<00:00,  1.05it/s]
Best Objective Value on Iteration 14: 0.003668603952974081
Best Index on Iteration 14: 9
Number of Neurons in layer 1 on Iteration 14: 887.0
Number of Neurons in layer 2 on Iteration 14: 887.0
________________________________

100%|██████████| 64/64 [00:59<00:00,  1.07it/s]
Best Objective Value on Iteration 15: 0.0037087001837790012
Best Index on Iteration 15: 48
Number of Neurons in layer 1 on Iteration 15: 879.0
Number of Neurons in layer 2 on Iteration 15: 879.0
________________________________

100%|██████████| 64/64 [01:01<00:00,  1.04it/s]
Best Objective Value on Iteration 16: 0.003759735031053424
Best Index on Iteration 16: 52
Number of Neurons in layer 1 on Iteration 16: 871.0
Number of Neurons in layer 2 on Iteration 16: 871.0
________________________________

100%|██████████| 64/64 [01:00<00:00,  1.05it/s]
Best Objective Value on Iteration 17: 0.0038104176055639982
Best Index on Iteration 17: 24
Number of Neurons in layer 1 on Iteration 17: 863.0
Number of Neurons in layer 2 on Iteration 17: 863.0
________________________________

100%|██████████| 64/64 [01:00<00:00,  1.06it/s]
Best Objective Value on Iteration 18: 0.0038675800897181034
Best Index on Iteration 18: 47
Number of Neurons in layer 1 on Iteration 18: 855.0
Number of Neurons in layer 2 on Iteration 18: 855.0
________________________________

100%|██████████| 64/64 [00:57<00:00,  1.11it/s]
Best Objective Value on Iteration 19: 0.003922298084944487
Best Index on Iteration 19: 18
Number of Neurons in layer 1 on Iteration 19: 847.0
Number of Neurons in layer 2 on Iteration 19: 847.0
________________________________

100%|██████████| 64/64 [01:01<00:00,  1.04it/s]
Best Objective Value on Iteration 20: 0.003985384479165077
Best Index on Iteration 20: 43
Number of Neurons in layer 1 on Iteration 20: 839.0
Number of Neurons in layer 2 on Iteration 20: 839.0
________________________________

100%|██████████| 64/64 [00:59<00:00,  1.08it/s]
Best Objective Value on Iteration 21: 0.004053016193211079
Best Index on Iteration 21: 3
Number of Neurons in layer 1 on Iteration 21: 831.0
Number of Neurons in layer 2 on Iteration 21: 831.0
________________________________

100%|██████████| 64/64 [00:56<00:00,  1.13it/s]
Best Objective Value on Iteration 22: 0.004124375060200691
Best Index on Iteration 22: 54
Number of Neurons in layer 1 on Iteration 22: 823.0
Number of Neurons in layer 2 on Iteration 22: 823.0
________________________________

100%|██████████| 64/64 [00:55<00:00,  1.15it/s]
Best Objective Value on Iteration 23: 0.004197209142148495
Best Index on Iteration 23: 32
Number of Neurons in layer 1 on Iteration 23: 815.0
Number of Neurons in layer 2 on Iteration 23: 815.0
________________________________

100%|██████████| 64/64 [00:57<00:00,  1.12it/s]
Best Objective Value on Iteration 24: 0.004277561791241169
Best Index on Iteration 24: 59
Number of Neurons in layer 1 on Iteration 24: 807.0
Number of Neurons in layer 2 on Iteration 24: 807.0
________________________________

100%|██████████| 64/64 [00:58<00:00,  1.10it/s]
Best Objective Value on Iteration 25: 0.004361486062407494
Best Index on Iteration 25: 32
Number of Neurons in layer 1 on Iteration 25: 799.0
Number of Neurons in layer 2 on Iteration 25: 799.0
________________________________

100%|██████████| 64/64 [01:05<00:00,  1.02s/it]
Best Objective Value on Iteration 26: 0.004435949493199587
Best Index on Iteration 26: 54
Number of Neurons in layer 1 on Iteration 26: 792.0
Number of Neurons in layer 2 on Iteration 26: 792.0
________________________________

100%|██████████| 64/64 [00:58<00:00,  1.10it/s]
Best Objective Value on Iteration 27: 0.004513287916779518
Best Index on Iteration 27: 38
Number of Neurons in layer 1 on Iteration 27: 785.0
Number of Neurons in layer 2 on Iteration 27: 785.0
________________________________

100%|██████████| 64/64 [00:57<00:00,  1.12it/s]
Best Objective Value on Iteration 28: 0.0045944941230118275
Best Index on Iteration 28: 37
Number of Neurons in layer 1 on Iteration 28: 778.0
Number of Neurons in layer 2 on Iteration 28: 778.0
________________________________

100%|██████████| 64/64 [00:58<00:00,  1.09it/s]
Best Objective Value on Iteration 29: 0.004681011196225882
Best Index on Iteration 29: 51
Number of Neurons in layer 1 on Iteration 29: 771.0
Number of Neurons in layer 2 on Iteration 29: 771.0
________________________________

100%|██████████| 64/64 [00:56<00:00,  1.14it/s]
Best Objective Value on Iteration 30: 0.0047669606283307076
Best Index on Iteration 30: 53
Number of Neurons in layer 1 on Iteration 30: 764.0
Number of Neurons in layer 2 on Iteration 30: 764.0
________________________________

100%|██████████| 64/64 [00:56<00:00,  1.12it/s]
Best Objective Value on Iteration 31: 0.004848496522754431
Best Index on Iteration 31: 6
Number of Neurons in layer 1 on Iteration 31: 757.0
Number of Neurons in layer 2 on Iteration 31: 757.0
________________________________

100%|██████████| 64/64 [00:53<00:00,  1.20it/s]
Best Objective Value on Iteration 32: 0.004950187634676695
Best Index on Iteration 32: 18
Number of Neurons in layer 1 on Iteration 32: 750.0
Number of Neurons in layer 2 on Iteration 32: 750.0
________________________________

100%|██████████| 64/64 [00:52<00:00,  1.22it/s]
Best Objective Value on Iteration 33: 0.005051175132393837
Best Index on Iteration 33: 4
Number of Neurons in layer 1 on Iteration 33: 743.0
Number of Neurons in layer 2 on Iteration 33: 743.0
________________________________

100%|██████████| 64/64 [00:54<00:00,  1.16it/s]
Best Objective Value on Iteration 34: 0.005157934036105871
Best Index on Iteration 34: 29
Number of Neurons in layer 1 on Iteration 34: 736.0
Number of Neurons in layer 2 on Iteration 34: 736.0
________________________________

100%|██████████| 64/64 [00:58<00:00,  1.10it/s]
Best Objective Value on Iteration 35: 0.00526472507044673
Best Index on Iteration 35: 61
Number of Neurons in layer 1 on Iteration 35: 729.0
Number of Neurons in layer 2 on Iteration 35: 729.0
________________________________

100%|██████████| 64/64 [00:55<00:00,  1.16it/s]
Best Objective Value on Iteration 36: 0.0053803944028913975
Best Index on Iteration 36: 9
Number of Neurons in layer 1 on Iteration 36: 722.0
Number of Neurons in layer 2 on Iteration 36: 722.0
________________________________

100%|██████████| 64/64 [00:52<00:00,  1.21it/s]
Best Objective Value on Iteration 37: 0.005498683545738459
Best Index on Iteration 37: 0
Number of Neurons in layer 1 on Iteration 37: 715.0
Number of Neurons in layer 2 on Iteration 37: 715.0
________________________________

100%|██████████| 64/64 [00:53<00:00,  1.20it/s]
Best Objective Value on Iteration 38: 0.005622826050966978
Best Index on Iteration 38: 29
Number of Neurons in layer 1 on Iteration 38: 708.0
Number of Neurons in layer 2 on Iteration 38: 708.0
________________________________

100%|██████████| 64/64 [00:55<00:00,  1.14it/s]
Best Objective Value on Iteration 39: 0.005730421282351017
Best Index on Iteration 39: 41
Number of Neurons in layer 1 on Iteration 39: 701.0
Number of Neurons in layer 2 on Iteration 39: 701.0
________________________________

100%|██████████| 64/64 [00:53<00:00,  1.19it/s]
Best Objective Value on Iteration 40: 0.005857251584529877
Best Index on Iteration 40: 7
Number of Neurons in layer 1 on Iteration 40: 694.0
Number of Neurons in layer 2 on Iteration 40: 694.0
________________________________

100%|██████████| 64/64 [00:52<00:00,  1.22it/s]
Best Objective Value on Iteration 41: 0.005973382852971554
Best Index on Iteration 41: 35
Number of Neurons in layer 1 on Iteration 41: 688.0
Number of Neurons in layer 2 on Iteration 41: 688.0
________________________________

100%|██████████| 64/64 [00:54<00:00,  1.17it/s]
Best Objective Value on Iteration 42: 0.006091853138059378
Best Index on Iteration 42: 41
Number of Neurons in layer 1 on Iteration 42: 682.0
Number of Neurons in layer 2 on Iteration 42: 682.0
________________________________

100%|██████████| 64/64 [00:55<00:00,  1.16it/s]
Best Objective Value on Iteration 43: 0.006213004235178232
Best Index on Iteration 43: 55
Number of Neurons in layer 1 on Iteration 43: 676.0
Number of Neurons in layer 2 on Iteration 43: 676.0
________________________________

100%|██████████| 64/64 [00:57<00:00,  1.12it/s]
Best Objective Value on Iteration 44: 0.006328057963401079
Best Index on Iteration 44: 43
Number of Neurons in layer 1 on Iteration 44: 670.0
Number of Neurons in layer 2 on Iteration 44: 670.0
________________________________

100%|██████████| 64/64 [00:56<00:00,  1.14it/s]
Best Objective Value on Iteration 45: 0.00645716767758131
Best Index on Iteration 45: 6
Number of Neurons in layer 1 on Iteration 45: 664.0
Number of Neurons in layer 2 on Iteration 45: 664.0
________________________________

100%|██████████| 64/64 [00:52<00:00,  1.22it/s]
Best Objective Value on Iteration 46: 0.006592088378965855
Best Index on Iteration 46: 60
Number of Neurons in layer 1 on Iteration 46: 658.0
Number of Neurons in layer 2 on Iteration 46: 658.0
________________________________

100%|██████████| 64/64 [00:52<00:00,  1.21it/s]
Best Objective Value on Iteration 47: 0.006727884523570538
Best Index on Iteration 47: 61
Number of Neurons in layer 1 on Iteration 47: 652.0
Number of Neurons in layer 2 on Iteration 47: 652.0
________________________________

100%|██████████| 64/64 [00:52<00:00,  1.22it/s]
Best Objective Value on Iteration 48: 0.006877876352518797
Best Index on Iteration 48: 38
Number of Neurons in layer 1 on Iteration 48: 646.0
Number of Neurons in layer 2 on Iteration 48: 646.0
________________________________

100%|██████████| 64/64 [00:58<00:00,  1.10it/s]
Best Objective Value on Iteration 49: 0.0070150247775018215
Best Index on Iteration 49: 54
Number of Neurons in layer 1 on Iteration 49: 640.0
Number of Neurons in layer 2 on Iteration 49: 640.0
________________________________

100%|██████████| 64/64 [00:53<00:00,  1.19it/s]
Best Objective Value on Iteration 50: 0.007162090390920639
Best Index on Iteration 50: 47
Number of Neurons in layer 1 on Iteration 50: 634.0
Number of Neurons in layer 2 on Iteration 50: 634.0
________________________________

100%|██████████| 64/64 [00:50<00:00,  1.26it/s]
Best Objective Value on Iteration 51: 0.00732030812650919
Best Index on Iteration 51: 18
Number of Neurons in layer 1 on Iteration 51: 628.0
Number of Neurons in layer 2 on Iteration 51: 628.0
________________________________

100%|██████████| 64/64 [00:50<00:00,  1.27it/s]
Best Objective Value on Iteration 52: 0.007485729176551104
Best Index on Iteration 52: 52
Number of Neurons in layer 1 on Iteration 52: 622.0
Number of Neurons in layer 2 on Iteration 52: 622.0
________________________________

100%|██████████| 64/64 [00:51<00:00,  1.25it/s]
Best Objective Value on Iteration 53: 0.007654916495084763
Best Index on Iteration 53: 49
Number of Neurons in layer 1 on Iteration 53: 616.0
Number of Neurons in layer 2 on Iteration 53: 616.0
________________________________

100%|██████████| 64/64 [00:53<00:00,  1.20it/s]
Best Objective Value on Iteration 54: 0.007826637476682663
Best Index on Iteration 54: 25
Number of Neurons in layer 1 on Iteration 54: 610.0
Number of Neurons in layer 2 on Iteration 54: 610.0
________________________________

100%|██████████| 64/64 [00:55<00:00,  1.16it/s]
Best Objective Value on Iteration 55: 0.008002998307347298
Best Index on Iteration 55: 52
Number of Neurons in layer 1 on Iteration 55: 604.0
Number of Neurons in layer 2 on Iteration 55: 604.0
________________________________

100%|██████████| 64/64 [00:54<00:00,  1.18it/s]
Best Objective Value on Iteration 56: 0.008181864395737648
Best Index on Iteration 56: 3
Number of Neurons in layer 1 on Iteration 56: 598.0
Number of Neurons in layer 2 on Iteration 56: 598.0
________________________________

100%|██████████| 64/64 [00:54<00:00,  1.17it/s]
Best Objective Value on Iteration 57: 0.008334318175911903
Best Index on Iteration 57: 44
Number of Neurons in layer 1 on Iteration 57: 593.0
Number of Neurons in layer 2 on Iteration 57: 593.0
________________________________

100%|██████████| 64/64 [00:54<00:00,  1.17it/s]
Best Objective Value on Iteration 58: 0.008477731607854366
Best Index on Iteration 58: 13
Number of Neurons in layer 1 on Iteration 58: 588.0
Number of Neurons in layer 2 on Iteration 58: 588.0
________________________________

100%|██████████| 64/64 [00:54<00:00,  1.18it/s]
Best Objective Value on Iteration 59: 0.00863884948194027
Best Index on Iteration 59: 35
Number of Neurons in layer 1 on Iteration 59: 583.0
Number of Neurons in layer 2 on Iteration 59: 583.0
________________________________

100%|██████████| 64/64 [00:51<00:00,  1.23it/s]
Best Objective Value on Iteration 60: 0.008803684264421463
Best Index on Iteration 60: 62
Number of Neurons in layer 1 on Iteration 60: 578.0
Number of Neurons in layer 2 on Iteration 60: 578.0
________________________________

100%|██████████| 64/64 [00:55<00:00,  1.16it/s]
Best Objective Value on Iteration 61: 0.008978716097772121
Best Index on Iteration 61: 58
Number of Neurons in layer 1 on Iteration 61: 573.0
Number of Neurons in layer 2 on Iteration 61: 573.0
________________________________

100%|██████████| 64/64 [00:52<00:00,  1.21it/s]
Best Objective Value on Iteration 62: 0.009153840132057667
Best Index on Iteration 62: 60
Number of Neurons in layer 1 on Iteration 62: 568.0
Number of Neurons in layer 2 on Iteration 62: 568.0
________________________________

100%|██████████| 64/64 [00:56<00:00,  1.14it/s]
Best Objective Value on Iteration 63: 0.009320097975432873
Best Index on Iteration 63: 44
Number of Neurons in layer 1 on Iteration 63: 563.0
Number of Neurons in layer 2 on Iteration 63: 563.0
________________________________

100%|██████████| 64/64 [00:56<00:00,  1.13it/s]
Best Objective Value on Iteration 64: 0.009495026431977749
Best Index on Iteration 64: 52
Number of Neurons in layer 1 on Iteration 64: 558.0
Number of Neurons in layer 2 on Iteration 64: 558.0
________________________________

100%|██████████| 64/64 [00:53<00:00,  1.21it/s]
Best Objective Value on Iteration 65: 0.009675626643002033
Best Index on Iteration 65: 22
Number of Neurons in layer 1 on Iteration 65: 553.0
Number of Neurons in layer 2 on Iteration 65: 553.0
________________________________

100%|██████████| 64/64 [00:55<00:00,  1.15it/s]
Best Objective Value on Iteration 66: 0.009858785197138786
Best Index on Iteration 66: 60
Number of Neurons in layer 1 on Iteration 66: 548.0
Number of Neurons in layer 2 on Iteration 66: 548.0
________________________________

100%|██████████| 64/64 [00:55<00:00,  1.15it/s]
Best Objective Value on Iteration 67: 0.010064566507935524
Best Index on Iteration 67: 21
Number of Neurons in layer 1 on Iteration 67: 543.0
Number of Neurons in layer 2 on Iteration 67: 543.0
________________________________